# This Notebook tests the importance of having proofread cells
## Compares the cumulative results of 2/3P pre-synaptic cells for various levels of cleanness
- **clean**: basic level of proofreading - only pre-synaptic cell only proofread around the soma
- **extended**: proofreading extends the entire length of the axon and dendrites. These cells (and clean cells) may still have errors in labeling which synapses are input vs. output, and there is probably not any proofreading of the targets.
- **manual cell typing (in column)**: This analysis is performed only on the synapses formed within a certain proofread column, which means many (as of 7/13, about a fifth) of the targets have been proofread as well, and ALL OF THEM HAVE BEEN MANUALLY CELL TYPED.

## Known proofreading-related issues/biases
Excitatory cells tend to be falsely merged more often than inhibitory cells because of their spines

**Are there biases in which cells have been proofread?**

# Manual vs. Automatic cell typing

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import warnings

In [2]:
cache_dir="..\\data\\"

In [10]:
results_dir = os.path.join(cache_dir, "results", "23P-clean-subclass-col")
manual_cumul_results_dir = os.path.join(results_dir, "manual-cell-typing", "cumulative")
auto_cumul_results_dir = os.path.join(results_dir, "auto-cell-typing", "cumulative")
with open(os.path.join(manual_cumul_results_dir, "4_clean_cells_cumulative.json")) as f:
    manual_results = json.loads(f.read())
with open(os.path.join(auto_cumul_results_dir, "4_clean_cells_cumulative.json")) as f:
    auto_results = json.loads(f.read())
manual_results.keys()

dict_keys(['parameters', 'p_post_syn_type_given_pre', 'p_type_by_path_dists', 'p_type_by_euc_dists', 'euc_hists', 'path_hists', 'euc_densities', 'path_densities'])

In [26]:
manual_n = 385  # = sum(sum(manual_results["euc_hists"][t].values()) for t in manual_results["euc_hists"]
manual_results["p_post_syn_type_given_pre"]  # n = 385

{'BC': 0.2025974025974026,
 'MC': 0.17662337662337663,
 'BPC': 0.015584415584415584,
 'NGC': 0.012987012987012988}

In [27]:
auto_n = 377  # = sum(sum(auto_results["euc_hists"][t].values()) for t in auto_results["euc_hists"]
auto_results["p_post_syn_type_given_pre"]  # n = 377

{'BC': 0.19363395225464192,
 'MC': 0.1962864721485411,
 'BPC': 0.015915119363395226,
 'NGC': 0.007957559681697613}

In [28]:
p = (manual_n - auto_n) / manual_n
p

0.02077922077922078

In [29]:
2 * np.sqrt(p * (1 - p) / manual_n)  # TODO correction because only 8 successes also this isn't even right because the cells may not even be the same. Use precision and recall

0.014539670681002079

There are 2.1% ($\pm $1.5) fewer neurons according to the nucleus neuron SVM model than reality. 

If you look at the entire column: Manual validation performed on a column of 1316 nuclei in Minnie65 measured a recall of .996 and a precision of .969.

In [24]:
auto_n_by_type = dict((t, auto_results["p_post_syn_type_given_pre"][t] * 377) for t in auto_results["p_post_syn_type_given_pre"])
auto_n_by_type

{'BC': 73.0, 'MC': 74.0, 'BPC': 6.0, 'NGC': 3.0}

In [25]:
manual_n_by_type = dict((t, manual_results["p_post_syn_type_given_pre"][t] * 385) for t in manual_results["p_post_syn_type_given_pre"])
manual_n_by_type

{'BC': 78.0, 'MC': 68.0, 'BPC': 6.0, 'NGC': 5.0}

In [34]:
for ctype in manual_results["p_post_syn_type_given_pre"]:
    type_p = abs(manual_n_by_type[ctype] - auto_n_by_type[ctype]) / manual_n_by_type[ctype]
    print(ctype, type_p, "pm", 2 * np.sqrt(type_p * (1 - type_p) / manual_n_by_type[ctype]))  # TODO correction because only 8 successes

BC 0.0641025641025641 pm 0.05546698041918816
MC 0.08823529411764706 pm 0.06879202543969219
BPC 0.0 pm 0.0
NGC 0.4 pm 0.4381780460041329
